# Facebook爬蟲
這裡簡單介紹一下爬蟲的邏輯再開始爬進入正題吧！

# 載入使用套件

In [1]:
import pandas as pd
import re, time, requests, datetime, gc, os
from selenium import webdriver
from bs4 import BeautifulSoup

# 經常用到的函數
Facebook請用戶「登入/註冊」的彈窗會反覆出現，在這裡先寫好嘗試點擊的函數提供使用

In [2]:
# 點擊不要現在註冊帳號
def ClickNotNow():
    try:
        driver.find_element_by_xpath('//a[@id="expanding_cta_close_button"]').click()
    except:
        time.sleep(0.1)

# 截取塗鴉墻貼文資訊並比對先前資料
如果每次都把所有貼文抓下來，無疑會浪費程式效能，在這裡我將載入先前的資料，並將各篇貼文分成以下三類：

- **新貼文**：截取貼文內容與留言
- **舊貼文但有新留言**：截取所有留言但只將新留言加入資料庫
- **舊貼文也無新留言**：不處理

可想而知，如果設定排程每天執行，大多數資料都會是「舊貼文也無新留言」的類型，加入這個判斷機制將大幅節省系統效能

In [3]:
# 截取塗鴉墻上貼文的留言數
def Wall_PostComments(i):
    try:
        Comments = i.find('a', {'data-testid':'UFI2CommentsCount/root'}).text.split(' ',2)[0]
        if 'K' in Comments:
            Comments = int(float(Comments.split('K')[0])*1000)
        else:
            Comments = int(Comments)
    except:
        Comments = 0
    return Comments

# 截取塗鴉墻上貼文的連結
def Wall_PostLink(i):
    Link = 'https://www.facebook.com' + i.find('a',{'class':'_5pcq'}).attrs['href'].split('?',2)[0]
    return Link

# 截取塗鴉墻上貼文的發佈時間
def Wall_PostTime(i):
    try:
        Time = i.find('abbr').attrs['title']
        Time = datetime.datetime.strptime(Time, '%m/%d/%y, %I:%M %p')
        Time = Time.strftime("%Y-%m-%d %H:%M")
    except:
        Time = 'Not Post'
    return Time

def CarwlList(urls, n, Posts):
    CheckList = pd.DataFrame(columns = ['Link','Time','Comments']) 
    for i in urls:
        driver.get(i)        
        for i in range(n):
            try:
                time.sleep(1)
                driver.find_element_by_css_selector('a.pam.uiBoxLightblue.uiMorePagerPrimary').click() # 加載更多貼文的按鈕
            except:
                time.sleep(1)
            # 這裡會跳出要我們登入的大畫面，找到「稍後再說」的按鈕並點擊
            ClickNotNow()
            soup = BeautifulSoup(driver.page_source)
            for i in soup.find_all('div', {'class':'_5pcr userContentWrapper'}):
                CheckList = pd.concat([CheckList, pd.DataFrame(data = [{'Link':Wall_PostLink(i),
                                                                        'Time':Wall_PostTime(i),
                                                                        'Comments':Wall_PostComments(i)}],
                                                               columns = ['Link','Time','Comments'])],
                                       ignore_index = True)
    
    CarwlList = pd.merge(left = CheckList,
                         right = Posts.loc[:,['Link','Comments']],
                         how='left',
                         on='Link',
                         suffixes=('_c', '_p'),
                         indicator=True)
    CarwlList = pd.concat([CarwlList.loc[CarwlList._merge == 'left_only', :], # 新貼文
                           CarwlList.loc[CarwlList.Comments_c > CarwlList.Comments_p, :]], # 舊貼文但有新留言
                          ignore_index=True)
    CarwlList = CarwlList.drop_duplicates(subset = 'Link',
                                          keep= 'first',
                                          inplace = False)
    return CarwlList

# 爬取特定貼文的內容與留言
## 展開貼文與留言

In [4]:
# 關閉彈窗
def ClosePopup():
    time.sleep(1)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(1)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    ClickNotNow()
    driver.find_element_by_xpath('//div[@class="_5pcr userContentWrapper"]//a[@data-testid="UFI2CommentsCount/root"]').click()
    time.sleep(1)
    ClickNotNow()
    time.sleep(0.5)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(0.5)
    ClickNotNow()
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(0.5)
    ClickNotNow()

# 選擇按照New或Oldest來顯示留言(選擇最相關或所有留言都不會會真的顯示所有留言)
def ClickOldest():
    time.sleep(1)
    driver.find_element_by_xpath('//a[@data-testid="UFI2ViewOptionsSelector/link"]').click()
    time.sleep(1)
    try:
        driver.find_element_by_partial_link_text('Comments shown in chronological order with the oldest comments at the top.').click()
    except:
        try:
            driver.find_element_by_partial_link_text('New comments and those with new replies appear at the top.').click()
        except:
            print('Plz, Check this post arragne type!')
    time.sleep(1)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    ClickNotNow()
    time.sleep(1)

# 偵測是否有「更多留言」（第一層），若有則點擊
def MoreComments():
    k = len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]'))
    # 卡住的次數超過100次就停止
    l = 0
    while (k != 0) & (l <= 300):
        print('Click More Comments： ' + str(l) + ' times.')
        try:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            # 若彈出註冊視窗點擊not now!
            ClickNotNow()
            driver.find_element_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]').click()
            time.sleep(1)
            k = len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]'))
        except:
            time.sleep(0.1)
        finally:
            l += 1
    time.sleep(1)

# 偵測是否有「更多回覆」（第二層），若有則點擊
def MoreReplies():
    k = len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]'))
    # 卡住的次數超過100次就停止
    l = 0
    while (k != 0) & (l <= 300):
        l += 1
        print('Click More Replies： ' + str(l) + ' times.')
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        driver.find_element_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]').click()
        time.sleep(0.5)
        k = len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]'))
    time.sleep(1)
    
# 偵測是否有「展開留言」（第二層留言內容），若有則點擊
def SeeMore():
    k = len(driver.find_elements_by_xpath('//a[@class="_5v47 fss"]'))
    # 卡住的次數超過100次就停止
    l = 0
    while (k != 0) & (l <= 1000):
        l += 1
        print('Click See More ： ' + str(l) + ' times.')
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        driver.find_element_by_xpath('//a[@class="_5v47 fss"]').click()
        time.sleep(0.5)
        k = len(driver.find_elements_by_xpath('//a[@class="_5v47 fss"]'))
    time.sleep(1)
    
# 組合以上函數，將貼文內容展開
def PostExpand():
    ClosePopup()
    # ClickOldest() # 展開留言會有無法加載更多留言的問題，暫時先不使用
    MoreComments()
    MoreReplies()
    SeeMore()

## 爬取貼文內容與留言資訊

In [5]:
# 貼文留言數
def Post_Name(userContent):
    return userContent.find('img').attrs['aria-label']

# 貼文留言數
def Post_ID(userContent):
    return userContent.find('a', {'class':'_5pb8 o_c3pynyi2g _8o _8s lfloat _ohe'}).attrs['href'].split('/?ref')[0].split('https://www.facebook.com/')[1]

# 貼文時間
def Post_Time(userContent):
    try:
        Time = userContent.find('abbr').attrs['title']
        Time = datetime.datetime.strptime(Time, '%m/%d/%y, %I:%M %p')
        Time = Time.strftime("%Y-%m-%d %H:%M")
    except:
        Time = "Error, Please check this post's condiction!" 
    return Time

# 貼文內容
def Post_Content(userContent):
    try:
        Content = userContent.find('div', {'class':'_5pbx userContent _3576'}).text
    except:
        Content = "There's No Text Content!"
    return Content

# 貼文留言數
def Post_Comments(userContent):
    try:
        CommentCounts = userContent.find('a', {'data-testid':'UFI2CommentsCount/root'}).text.split(' ',2)[0]
        if 'K' in CommentCounts:
            CommentCounts = int(float(CommentCounts.split('K')[0])*1000)
        else:
            CommentCounts = int(CommentCounts)
    except:
        CommentCounts = 0 
    return CommentCounts

# 貼文分享數
def Post_Shares(userContent):
    try:
        ShareCounts = userContent.find('span', {'class':'_355t _4vn2'}).text.split(' ',2)[0]
        if 'K' in ShareCounts:
            ShareCounts = int(float(ShareCounts.split('K')[0])*1000)
        else:
            ShareCounts = int(ShareCounts)
    except:
        ShareCounts = 0
    return ShareCounts

# 貼文按讚心情數
def Post_Likes(userContent):
    try:
        LikeCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_LIKE'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in LikeCounts:
            LikeCounts = int(float(LikeCounts.split('K')[0])*1000)
        else:
            LikeCounts = int(LikeCounts)
    except:
        LikeCounts = 0
    return LikeCounts

# 貼文愛心心情數
def Post_Loves(userContent):
    try:
        LoveCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_LOVE'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in LoveCounts:
            LoveCounts = int(float(LoveCounts.split('K')[0])*1000)
        else:
            LoveCounts = int(LoveCounts)
    except:
        LoveCounts = 0
    return LoveCounts

# 貼文哈哈心情數
def Post_Hahas(userContent):
    try:
        HahaCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_HAHA'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in HahaCounts:
            HahaCounts = int(float(HahaCounts.split('K')[0])*1000)
        else:
            HahaCounts = int(HahaCounts)
    except:
        HahaCounts = 0
    return HahaCounts

# 貼文Wow心情數
def Post_Wows(userContent):
    try:
        WowCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_WOW'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in WowCounts:
            WowCounts = int(float(WowCounts.split('K')[0])*1000)
        else:
            WowCounts = int(WowCounts)
    except:
        WowCounts = 0
    return WowCounts

# 貼文Sad心情數
def Post_Sads(userContent):
    try:
        SadCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_SORRY'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in SadCounts:
            SadCounts = int(float(SadCounts.split('K')[0])*1000)
        else:
            SadCounts = int(SadCounts)
    except:
        SadCounts = 0
    return SadCounts

# 貼文Angry心情數
def Post_Angrys(userContent):
    try:
        AngryCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_ANGER'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in AngryCounts:
            AngryCounts = int(float(AngryCounts.split('K')[0])*1000)
        else:
            AngryCounts = int(AngryCounts)
    except:
        AngryCounts = 0
    return AngryCounts

# 文章內容與互動摘要
def PostInfo(soup):
    # 貼文區
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    PostContent = pd.DataFrame(data = [{'Name':Post_Name(userContent),
                                        'ID':Post_ID(userContent),
                                        'Time':Post_Time(userContent),
                                        'Content':Post_Content(userContent),
                                        'Comments':Post_Comments(userContent),
                                        'Shares':Post_Shares(userContent),
                                        'Likes':Post_Likes(userContent),
                                        'Loves':Post_Loves(userContent),
                                        'Hahas':Post_Hahas(userContent),
                                        'Wows':Post_Wows(userContent),
                                        'Sads':Post_Sads(userContent),
                                        'Angrys':Post_Angrys(userContent),
                                        'Updatetime':datetime.datetime.now().strftime("%Y-%m-%d %H:%M"),
                                        'Link':driver.current_url}],
                            columns = ['Name', 'ID', 'Time', 'Content', 'Comments', 'Shares', 'Likes', 'Loves', 'Hahas', 'Wows', 'Sads', 'Angrys', 'Updatetime', 'Link'])
    return PostContent

# 留言內容
def Comment_Content(element):
    try:
        Content = element.find('span', {'dir':'ltr'}).text
    except:
        Content = 'img'
    return Content

def CommentsInfo(soup):  
    PostComments = pd.DataFrame()
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    try:
        for i in userContent.select('ul._7a9a > li'):
            # 先抓留言並放在Comment
            Comment = pd.DataFrame(data=[{'ID':i.find('a', {'class':' _3mf5 _3mg0'}).attrs['data-hovercard'].split('id=',2)[1],
                                          'Name':i.find('img').attrs['alt'],
                                          'Time':datetime.datetime.strptime(i.find('abbr',{'class':'livetimestamp'}).attrs['data-tooltip-content'], '%A, %B %d, %Y at %I:%M %p').strftime("%Y-%m-%d %H:%M"),
                                          'Content':Comment_Content(i),
                                          'RepID':userContent.find('div', {'class':'_5pcp _5lel _2jyu _232_'}).attrs['id'].split(';')[0].split('feed_subtitle_')[-1],
                                          'RepName':userContent.find('img').attrs['aria-label'],
                                          'Link':driver.current_url}],
                                   columns = ['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName', 'Link'])
            PostComments = pd.concat([PostComments, Comment], ignore_index=True)
            # 留言的留言
            for j in i.findAll('div', {'data-testid':'UFI2Comment/root_depth_1'}):
                Comment = pd.DataFrame(data=[{'ID':j.find('a', {'class':' _3mf5 _3mg1'}).attrs['data-hovercard'].split('id=',2)[1],
                                              'Name':j.find('img').attrs['alt'],
                                              'Time':datetime.datetime.strptime(j.find('abbr',{'class':'livetimestamp'}).attrs['data-tooltip-content'], '%A, %B %d, %Y at %I:%M %p').strftime("%Y-%m-%d %H:%M"),
                                              'Content':Comment_Content(j),
                                              'RepID':i.find('a', {'class':' _3mf5 _3mg0'}).attrs['data-hovercard'].split('id=',2)[1],
                                              'RepName':i.find('img').attrs['alt'],
                                              'Link':driver.current_url}],
                                       columns = ['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName', 'Link'])
                PostComments = pd.concat([PostComments, Comment], ignore_index=True)
        PostComments['Updatetime'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
    except:
        print('Crawl Comments Failed!')
    return PostComments

# 更新貼文與留言資訊
將最新抓到的貼文併回Post表格，並以ID與Time作為關鍵(key)值，取最新抓取的資料(Updatetime)

In [6]:
def UpdateData(DateFrame_o,DateFrame_n):
    DataFrame = pd.concat([DateFrame_o, DateFrame_n], ignore_index=True)
    DataFrame = DataFrame.sort_values(by = 'Updatetime', ascending = False)
    DataFrame = DataFrame.drop_duplicates(subset = ['ID', 'Time', 'Content'],
                                          keep= 'first',
                                          inplace = False)
    return DataFrame

# 爬資料
## 讀取先前資料
若在桌面偵測不到檔案，就創造新的dataframe

In [7]:
try:
    Posts = pd.read_pickle('C:/Users/' + os.getlogin() + '/Desktop/Posts.plk')
except:
    Posts = pd.DataFrame(columns=['Name', 'ID', 'Time', 'Content', 'Comments', 'Shares', 'Likes', 'Loves', 'Hahas', 'Wows', 'Sads', 'Angrys', 'Updatetime', 'Link'])
try:
    Comments = pd.read_pickle('C:/Users/' + os.getlogin() + '/Desktop/Comments.plk')
except:
    Comments = pd.DataFrame(columns = ['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName', 'Link', 'Updatetime'])

print(Posts.shape)
print(Comments.shape)
urls = ['https://www.facebook.com/tstartel/',
        'https://www.facebook.com/chtmobile/',
        'https://www.facebook.com/taiwanmobile/',
        'https://www.facebook.com/fareastone/',
        'https://www.facebook.com/Aptg.tw/']

(1494, 14)
(222578, 8)


## 產出待爬清單

In [8]:
driver = webdriver.Chrome()
driver.get('https://www.facebook.com/')
time.sleep(1)
driver.find_element_by_partial_link_text('English').click()

In [9]:
CarwlList = CarwlList(urls=urls, n=10, Posts = Posts)
CarwlList

,Link,Time,Comments_c,Comments_p,_merge
0,https://www.facebook.com/tstartel/photos/a.413...,2019-06-08 21:00,39,NaN,left_only
1,https://www.facebook.com/tstartel/posts/333571...,2019-06-08 00:50,60,NaN,left_only
2,https://www.facebook.com/tstartel/posts/333077...,2019-06-07 02:00,57,NaN,left_only
3,https://www.facebook.com/tstartel/photos/a.413...,2019-06-06 21:00,38,NaN,left_only
4,https://www.facebook.com/tstartel/videos/35191...,2019-06-06 07:19,66,NaN,left_only
5,https://www.facebook.com/tstartel/photos/a.413...,2019-06-06 02:33,219,NaN,left_only
6,https://www.facebook.com/tstartel/posts/333045...,2019-06-06 02:18,23,NaN,left_only
14,https://www.facebook.com/tstartel/photos/a.413...,2019-06-05 22:00,569,NaN,left_only
79,https://www.facebook.com/chtmobile/posts/27601...,2019-06-05 21:05,2,NaN,left_only
82,https://www.facebook.com/chtmobile/posts/27573...,2019-06-05 04:00,0,NaN,left_only


如同上面的說明，當CheckLists「merge」欄位為「left_only」時表示該貼文是新貼文；

而當「merge」欄位為「both」，但「Comments_c」的值大於「Comments_p」時，表示該貼文是舊資料但有新留言。

以下將截取這兩類的貼文作為爬取清單

## 執行爬蟲

In [10]:
for i in CarwlList.Link:
    print('Dealing with: ' + i)
    driver.get(i)
    try:
        PostExpand()
        print('Expand Succed!')
        try:
            soup = BeautifulSoup(driver.page_source)
            time.sleep(1)
            nPost = PostInfo(soup)
            nComments = CommentsInfo(soup)
            Posts = UpdateData(DateFrame_o = Posts, DateFrame_n = nPost)
            print('Update PostInfo complete!')
            Comments = UpdateData(DateFrame_o = Comments,DateFrame_n = nComments)
            print('Update CommentsInfo complete!')
        except:
            print('Update Post or CommentsInfo Failed!')
    except:
        print('This post can not expand')
    gc.collect()
    print('Time Log: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + '\n\n------------------')

Dealing with: https://www.facebook.com/tstartel/photos/a.413576055335114/3325555817470442/
Click More Comments： 0 times.
Click More Replies： 1 times.
Click More Replies： 2 times.
Click More Replies： 3 times.
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-06-10 11:50:53

------------------
Dealing with: https://www.facebook.com/tstartel/posts/3335717143120976
Expand Succed!
Update PostInfo complete!


C:\Users\TL_Yu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


Update CommentsInfo complete!
Time Log: 2019-06-10 11:51:15

------------------
Dealing with: https://www.facebook.com/tstartel/posts/3330770210282336
This post can not expand
Time Log: 2019-06-10 11:51:18

------------------
Dealing with: https://www.facebook.com/tstartel/photos/a.413576055335114/3330531623639528/
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-06-10 11:51:40

------------------
Dealing with: https://www.facebook.com/tstartel/videos/351918758857489/
Click More Comments： 0 times.
Click More Replies： 1 times.
Click More Replies： 2 times.
Click More Replies： 3 times.
Click More Replies： 4 times.
Click More Replies： 5 times.
Click More Replies： 6 times.
Click More Replies： 7 times.
Click See More ： 1 times.
Click See More ： 2 times.
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-06-10 11:52:59

------------------
Dealing with: https://www.facebook.com/tstartel/photos/a.413576055335114/33304450003

Click More Comments： 0 times.
Click More Comments： 1 times.
Click More Comments： 2 times.
Click More Comments： 3 times.
Click More Comments： 4 times.
Click More Comments： 5 times.
Click More Comments： 6 times.
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-06-10 11:59:42

------------------
Dealing with: https://www.facebook.com/tstartel/posts/3327691170590240
Click More Comments： 0 times.
Click More Replies： 1 times.
Click More Replies： 2 times.
Click More Replies： 3 times.
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-06-10 11:59:58

------------------
Dealing with: https://www.facebook.com/tstartel/photos/a.413576055335114/3327111277314896/
Click More Comments： 0 times.
Click More Replies： 1 times.
Click More Replies： 2 times.
Click More Replies： 3 times.
Click More Replies： 4 times.
Click More Replies： 5 times.
Click See More ： 1 times.
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complet

Click More Comments： 0 times.
Click More Comments： 1 times.
Click More Comments： 2 times.
Click More Comments： 3 times.
Click More Comments： 4 times.
Click More Comments： 5 times.
Click More Comments： 6 times.
Click More Comments： 7 times.
Click More Comments： 8 times.
Click More Comments： 9 times.
Click More Comments： 10 times.
Click More Comments： 11 times.
Click More Comments： 12 times.
Click More Replies： 1 times.
Click More Replies： 2 times.
Click More Replies： 3 times.
Click More Replies： 4 times.
Click More Replies： 5 times.
Click More Replies： 6 times.
Click More Replies： 7 times.
Click More Replies： 8 times.
Click More Replies： 9 times.
Click More Replies： 10 times.
Click More Replies： 11 times.
Click See More ： 1 times.
Click See More ： 2 times.
Click See More ： 3 times.
Click See More ： 4 times.
Click See More ： 5 times.
Click See More ： 6 times.
Click See More ： 7 times.
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-06-10 12:06:16

--

In [11]:
driver.quit()

In [12]:
Posts = Posts.reset_index(drop=True)
Posts = Posts.loc[:,['Name', 'ID', 'Time', 'Content', 'Comments', 'Shares', 'Likes', 'Loves', 'Hahas', 'Wows', 'Sads', 'Angrys', 'Updatetime', 'Link']]
Posts

,Name,ID,Time,Content,Comments,Shares,Likes,Loves,Hahas,Wows,Sads,Angrys,Updatetime,Link
0,遠傳電信,fareastone,2019-06-02 21:20,秀出你的粉紅少女心 #上傳照片抽Reno📱 💗帶點迷幻又不失可愛的粉嫩 💗不管多遠你都...,1100,1100,1200,6,0,0,0,0,2019-06-10 12:06,https://www.facebook.com/fareastone/photos/a.1...
1,遠傳電信,fareastone,2019-06-04 20:50,【遠傳網路門市 6月限定超殺優惠】 想換手機？不要再猶豫啦！！ 當夏就出手，好康才會跟著走~...,44,6,74,1,0,0,0,0,2019-06-10 12:06,https://www.facebook.com/fareastone/posts/2569...
2,中華電信行動學園,chtmobile,2019-05-29 02:00,【端午節會回家嗎？】 開完會，手邊的工作終於告一段落。 有三通未接來電，是媽媽。 「嗨媽，我...,18,4,64,2,0,0,0,0,2019-06-10 12:05,https://www.facebook.com/chtmobile/photos/a.26...
3,台灣之星,tstartel,2019-05-26 21:00,本月最強🔥全民一起辦$299/$499🔥 新申辦/攜碼 #4G上網+#網內通話 #雙飽 最後...,584,9,162,5,0,4,0,0,2019-06-10 12:04,https://www.facebook.com/tstartel/videos/23007...
4,台灣之星,tstartel,2019-05-04 20:57,【辦門號不吃虧 Ep.3】 這張照片怎麼賴給爸爸❓ 妹妹可以從美國打賴給我喔⁉️ 自從長輩也...,63,4,326,5,0,6,0,0,2019-06-10 12:04,https://www.facebook.com/tstartel/photos/a.413...
5,台灣之星,tstartel,2019-05-29 05:00,💥我們重視你的聲音 把選擇權還給你💥 #選出你的理想方案 #方案與你共同決策 來～故事是這樣...,720,314,623,8,0,2,0,0,2019-06-10 12:03,https://www.facebook.com/tstartel/posts/330880...
6,台灣之星,tstartel,2019-06-03 02:38,💥就在明天！與粉絲共同產出的資費來了💥 #我們重視你的聲音 #把選擇權還給你 🔖給屬於青春U...,205,6,223,4,4,0,0,0,2019-06-10 12:02,https://www.facebook.com/tstartel/photos/a.413...
7,台灣之星,tstartel,2019-06-03 20:30,還記得之前和大家分享iOS的披露消息嗎？ 正式消息終於在今天凌晨公開啦‼ 在眾多更新中，大家...,46,1,152,3,0,0,0,2,2019-06-10 12:01,https://www.facebook.com/tstartel/posts/332434...
8,台灣之星,tstartel,2019-06-03 04:53,台灣之星積極深耕在地、滿足不同需求😃 ▪北部限定 #挺北漂青年專案 http://bit.l...,299,20,601,10,7,0,0,0,2019-06-10 12:01,https://www.facebook.com/tstartel/photos/a.854...
9,台灣之星,tstartel,2019-06-04 21:00,天氣越來越熱🔥 #U姐編 每天都被烤得不要不要😵 立馬來杯透心涼的飲料是少不了 趁著 #世界...,36,1,128,7,0,0,0,1,2019-06-10 12:00,https://www.facebook.com/tstartel/photos/a.413...


In [13]:
Comments = Comments.reset_index(drop=True)
Comments = Comments.loc[:,['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName', 'Link', 'Updatetime']]
Comments

,ID,Name,Time,Content,RepID,RepName,Link,Updatetime
0,561647139,黃紫晴,2019-06-09 22:18,我的粉色帽子超搭#遠傳獨家#OPPOReno薄霧粉 Mike Yang 林憶萱 Susan ...,162583067088337,遠傳電信,https://www.facebook.com/fareastone/photos/a.1...,2019-06-10 12:06
1,100013725619537,Alma Chen,2019-06-04 20:40,Johnny Jiang Chang Dennis Allen Chang【我的粉色杯墊...,162583067088337,遠傳電信,https://www.facebook.com/fareastone/photos/a.1...,2019-06-10 12:06
2,100000361385257,Ya Mei Liao,2019-06-03 15:14,【我的粉色KITTY鞋超搭 #遠傳獨家 #OPPOReno薄霧粉】 Ryan Liao Ha...,162583067088337,遠傳電信,https://www.facebook.com/fareastone/photos/a.1...,2019-06-10 12:06
3,100004319610861,Ben Journey,2019-06-07 16:49,Dino Lay Li Chink Jason Cheah 【我的粉色窗簾超搭 #遠傳獨家 ...,162583067088337,遠傳電信,https://www.facebook.com/fareastone/photos/a.1...,2019-06-10 12:06
4,100006852821810,Downey Linken,2019-06-07 07:24,Robert Cai Amanda Emilia 藍諾雷【我的粉色菜刀超搭 #遠傳獨家 #O...,162583067088337,遠傳電信,https://www.facebook.com/fareastone/photos/a.1...,2019-06-10 12:06
5,100032167689138,李小艾,2019-06-04 10:30,Mos EightysixYU Tung@Hong Yun【我的粉色咖啡杯 超搭 #遠傳獨家...,162583067088337,遠傳電信,https://www.facebook.com/fareastone/photos/a.1...,2019-06-10 12:06
6,100004582596624,Mei Mei,2019-06-04 20:44,Jane Huang Hee Wang Tseng Gina【我的粉色抱枕 超搭 #遠傳獨...,162583067088337,遠傳電信,https://www.facebook.com/fareastone/photos/a.1...,2019-06-10 12:06
7,100000810274376,Chen David,2019-06-08 05:10,吳小美 Annabelle Wu Cheng Crown【我的粉色提袋超搭 ＃遠傳獨家 #O...,162583067088337,遠傳電信,https://www.facebook.com/fareastone/photos/a.1...,2019-06-10 12:06
8,100006895568082,Robert Cai,2019-06-07 11:21,Jojo Low Shen Lele Baratai Band Chai【我的粉色洗衣精超搭...,162583067088337,遠傳電信,https://www.facebook.com/fareastone/photos/a.1...,2019-06-10 12:06
9,100004077983046,潘達蹦,2019-06-07 10:58,Yingcch Chiang Mutiara Anggraini 張凱莉 【我的粉色乳液超搭...,162583067088337,遠傳電信,https://www.facebook.com/fareastone/photos/a.1...,2019-06-10 12:06


# 保存資料

In [14]:
Posts.to_pickle('C:/Users/' + os.getlogin() + '/Desktop/Posts.plk')
Comments.to_pickle('C:/Users/' + os.getlogin() + '/Desktop/Comments.plk')
Posts.to_csv('C:/Users/' + os.getlogin() + '/Desktop/Posts.csv')
Comments.to_csv('C:/Users/' + os.getlogin() + '/Desktop/Comments.csv')

# To Do List
- 故障(無法expand的貼文要避免加入貼文清單或者加入後，要讓下次再來爬文，不能直接跳過。)
 - 評估將加載更多留言放在同一個function中
 
 https://blog.csdn.net/qq_24499417/article/details/81408655

# 維護測試區

In [60]:
#driver = webdriver.Chrome()
#driver.get('https://www.facebook.com/')
#time.sleep(1)
#driver.find_element_by_partial_link_text('English').click()

In [9]:
PostExpand()

Click More Comments： 0 times.
Click More Comments： 1 times.
Click More Comments： 2 times.
Click More Comments： 3 times.
Click More Comments： 4 times.
Click More Comments： 5 times.
Click More Comments： 6 times.
Click More Comments： 7 times.
Click More Comments： 8 times.
Click More Replies： 1 times.
Click More Replies： 2 times.
Click More Replies： 3 times.
Click More Replies： 4 times.
Click More Replies： 5 times.
Click More Replies： 6 times.
Click More Replies： 7 times.
Click More Replies： 8 times.
Click More Replies： 9 times.
Click More Replies： 10 times.
Click More Replies： 11 times.
Click More Replies： 12 times.
Click More Replies： 13 times.
Click More Replies： 14 times.
Click More Replies： 15 times.
Click More Replies： 16 times.
Click More Replies： 17 times.
Click More Replies： 18 times.
Click More Replies： 19 times.
Click More Replies： 20 times.
Click More Replies： 21 times.
Click See More ： 1 times.
Click See More ： 2 times.


In [10]:
soup = BeautifulSoup(driver.page_source)
PostInfo(soup)

,Name,ID,Time,Content,Comments,Shares,Likes,Loves,Hahas,Wows,Sads,Angrys,Updatetime,Link
0,台灣之星,tstartel,2019-06-05 22:00,端午節了耶，你們愛南部粽還是北部粽? #週星星編 當然是南部粽呀❗❗ 北部粽根本是油飯穿粽葉...,558,133,1000,13,4,0,0,0,2019-06-10 00:02,https://www.facebook.com/tstartel/photos/a.413...


In [11]:
CommentsInfo(soup)

,ID,Name,Time,Content,RepID,RepName,Link,Updatetime
0,360044337354953,台灣之星,2019-06-06 13:00,活動時間： 即日起至6/9(日)23:59止活動獎項： 100元全家禮券x5名 抽獎辦法： ...,360044337354953,台灣之星,https://www.facebook.com/tstartel/photos/a.413...,2019-06-10 00:02
1,100000559662494,Tom Ling,2019-06-06 13:04,Michelle Wu ！端午節要幹嘛？當然是玩賓果和中午立蛋,360044337354953,台灣之星,https://www.facebook.com/tstartel/photos/a.413...,2019-06-10 00:02
2,100011295641038,黃婉玲,2019-06-06 13:05,莫陞芢！端午節要幹嘛？當然是玩賓果和中午立蛋呀,360044337354953,台灣之星,https://www.facebook.com/tstartel/photos/a.413...,2019-06-10 00:02
3,100004456059438,章原,2019-06-06 13:06,Christina Kao！端午節要幹嘛？當然是玩賓果和中午立蛋呀.,360044337354953,台灣之星,https://www.facebook.com/tstartel/photos/a.413...,2019-06-10 00:02
4,100000234074406,黃栯栯,2019-06-06 13:08,Lenny Chang！端午節要幹嘛？當然是玩賓果和看划龍舟呀,360044337354953,台灣之星,https://www.facebook.com/tstartel/photos/a.413...,2019-06-10 00:02
5,100000654610693,Li-Wan Chiu,2019-06-06 13:09,連佩婷 !端午節要幹嘛? 當然是玩賓果和懷念一下屈原呀,360044337354953,台灣之星,https://www.facebook.com/tstartel/photos/a.413...,2019-06-10 00:02
6,100003303399116,黃荷珺,2019-06-06 13:14,Hanxian Huang！端午節要幹嘛？當然是玩賓果和佩戴香包呀,360044337354953,台灣之星,https://www.facebook.com/tstartel/photos/a.413...,2019-06-10 00:02
7,100004587102723,Jia Yi,2019-06-06 13:15,Yifan Hu！端午節要幹嘛？當然是玩賓果和佩戴香包呀,360044337354953,台灣之星,https://www.facebook.com/tstartel/photos/a.413...,2019-06-10 00:02
8,100000133092219,UN Rgy,2019-06-06 13:16,端午節要幹嘛?可以在家睡覺嗎,360044337354953,台灣之星,https://www.facebook.com/tstartel/photos/a.413...,2019-06-10 00:02
9,360044337354953,台灣之星,2019-06-06 13:24,還要記得上傳完成自己的賓果串連，就有機會獲得 #抽100元全家禮券唷 ❤,100000133092219,UN Rgy,https://www.facebook.com/tstartel/photos/a.413...,2019-06-10 00:02


In [12]:
pd.read_pickle('C:/Users/' + os.getlogin() + '/Desktop/Comments.plk')

,ID,Name,Time,Content,RepID,RepName,Link,Updatetime
0,100008036701674,陳國進,2019-05-29 17:27,哇~是郭台銘,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/posts/2478267...,2019-06-06 10:26
1,100002470664192,Chun Ju Wu,2019-05-28 21:15,"合約到繼續用吧，199=199,299=299",162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-06-06 10:26
2,162174107155573,亞太電信Gt智慧生活,2019-06-04 19:33,您好，若您現在的手機門號是屬於無合約的狀態，可以直接申辦188方案喔！,100024104980789,孫天龍,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-06-06 10:26
3,100000198195362,蔡文豪,2019-05-31 10:31,老客戶已沒有綁約，不能換約嗎？,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-06-06 10:26
4,100000173438302,余雅慧,2019-06-01 17:47,申辦期間到何時？,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-06-06 10:26
5,162174107155573,亞太電信Gt智慧生活,2019-06-03 13:37,您好，此方案的申辦期間到6/30喔方案說明請參考：https://reurl.cc/AbAn3,100000173438302,余雅慧,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-06-06 10:26
6,100000701465888,Jimo Luo,2019-05-31 07:55,有支援亞太全頻的手機。在我家很猛 4頻全有。 遠傳室內連3mb都沒 只有700頻段收的到。亞...,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-06-06 10:26
7,100000818895234,蘇文雅,2019-05-30 11:00,實體門市台南有哪家。 原是台哥大用戶 6月中合約才到期 這樣能申辦嗎,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-06-06 10:26
8,162174107155573,亞太電信Gt智慧生活,2019-05-30 12:42,您好，299方案目前只限定網路門市辦理哦~方案到5/31止，需提前與原電信解約後再進行申辦喔！,100000818895234,蘇文雅,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-06-06 10:26
9,100000818895234,蘇文雅,2019-05-30 17:14,亞太電信Gt智慧生活 解約後不就不能用同一個號碼了,100000818895234,蘇文雅,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-06-06 10:26


In [13]:
UpdateData(DateFrame_o = pd.read_pickle('C:/Users/' + os.getlogin() + '/Desktop/Comments.plk'),
           DateFrame_n = CommentsInfo(soup))

,ID,Name,Time,Content,RepID,RepName,Link,Updatetime
223123,100004023503936,張絮花,2019-06-09 23:10,緹花端午節要幹嘛？當然是玩賓果和吃粽子阿,360044337354953,台灣之星,https://www.facebook.com/tstartel/photos/a.413...,2019-06-10 00:04
222757,100006394413602,林良政,2019-06-06 18:59,Lillian Owen！端午節要幹嘛？當然是玩賓果和中午立蛋,360044337354953,台灣之星,https://www.facebook.com/tstartel/photos/a.413...,2019-06-10 00:04
222763,360044337354953,台灣之星,2019-06-06 19:16,💛記得要上傳完成自己的賓果串連喔😊,1472728009,Lin Xiaojie,https://www.facebook.com/tstartel/photos/a.413...,2019-06-10 00:04
222762,1472728009,Lin Xiaojie,2019-06-06 19:05,Yi Zhen Lee端午節要幹嘛？當然是玩賓果和 佩戴香包,360044337354953,台灣之星,https://www.facebook.com/tstartel/photos/a.413...,2019-06-10 00:04
222761,100027559293655,Chen Chen,2019-06-06 19:04,JJ Joe Hsuan Hsuan端午節要幹嘛？當然是玩賓果和中午立蛋,360044337354953,台灣之星,https://www.facebook.com/tstartel/photos/a.413...,2019-06-10 00:04
222760,360044337354953,台灣之星,2019-06-06 19:13,💛記得要上傳完成自己的賓果串連喔😊,100002954847523,Wind Rain,https://www.facebook.com/tstartel/photos/a.413...,2019-06-10 00:04
222759,100002954847523,Wind Rain,2019-06-06 19:01,李銘峰！端午節要幹嘛？當然是玩賓果和 佩戴香包,360044337354953,台灣之星,https://www.facebook.com/tstartel/photos/a.413...,2019-06-10 00:04
222758,100029765175441,吳世華,2019-06-06 19:00,吳世忠端午節要幹嘛？當然是玩賓果和中午立蛋,360044337354953,台灣之星,https://www.facebook.com/tstartel/photos/a.413...,2019-06-10 00:04
222756,360044337354953,台灣之星,2019-06-06 19:03,記得找一位家人朋友~~還要上傳完成自己的賓果串連，就有機會獲得 #抽100元全家禮券唷❤,100012083079048,李春梅,https://www.facebook.com/tstartel/photos/a.413...,2019-06-10 00:04
222714,100002428678198,林中偉,2019-06-06 18:30,吳基福端午節要幹嘛？當然是玩賓果和吃肉粽呀,360044337354953,台灣之星,https://www.facebook.com/tstartel/photos/a.413...,2019-06-10 00:04


In [61]:
#driver.get('https://www.facebook.com/tstartel/videos/1100239263492734/?permPage=1')
#time.sleep(0.7)
#PostExpand()

# 其他

In [18]:
# 原本想要登入的方式獲取連結，但FB會偵測異常行為，暫時先不登入
# 關閉Chrome的「通知」提醒
#chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument("--disable-notifications")

#driver = webdriver.Chrome(options=chrome_options)
#url = 'https://www.facebook.com/'
#driver.get(url)
#time.sleep(3)

# 啟動瀏覽器並登入Facebook
#username = driver.find_element_by_id('email')
#username.send_keys('61034b001@gms.ndhu.edu.tw')
#passwd=driver.find_element_by_id('pass')
#passwd.send_keys('19920309')
#button=driver.find_element_by_id('loginbutton')
#button.click()